# CFG

In [ ]:
# GCS_PATH='../input/nbme-score-clinical-patient-notes/' # running on kaggle
GCS_PATH='../workspace/data/' # running on local machine

In [3]:
class CFG:
    num_workers=4
    # path="../input/deberta-v3-large-5-folds-public/" # running on Kaggle
    path = "/workspace/deberta-v3-large-5-folds-public" # running on local machine
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
#    batch_size=128
    batch_size=32
    fc_dropout=0.2
    max_len=354
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
#    trn_fold=[0]
    pl_folds=5
    pl_n_fold=1
    
    NO_ZERO=False

In [4]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

# input_dir = Path("../input/deberta-v2-3-fast-tokenizer") # running on Kaggle
input_dir = Path("/workspace/deberta-v2-3-fast-tokenizer") # running on local machine

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.7/site-packages/transformers'

In [5]:
import os
import gc
import ast
import sys
import copy
import json
import math
import string
import pickle
import random
import itertools
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

debug=False

env: TOKENIZERS_PARALLELISM=true
cuda


# Utils

In [6]:
def seed_everything(seed=42):
    '''
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    '''
    random.seed(seed)
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # When running on the CuDNN backend, two further options must be set
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(seed=CFG.seed)

# Tokenizer

In [ ]:
from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast

# tokenizer = DebertaV2TokenizerFast.from_pretrained('../input/deberta-tokenizer') # running on Kaggle
tokenizer = DebertaV2TokenizerFast.from_pretrained('../workspace/deberta-tokenizer') # running on local machine
CFG.tokenizer = tokenizer

# Helper functions for scoring

In [8]:
def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    
    return f1_score(truths, preds)


def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
        
    return binary


def span_micro_f1(preds, truths):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
        
    return micro_f1(bin_preds, bin_truths)

In [9]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
        
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
            
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
        
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
        
    return predictions


def get_score(y_true, y_pred):
    return span_micro_f1(y_true, y_pred)

# OOF

In [10]:
oof = pd.read_pickle(CFG.path+'oof_df.pkl')
truths = create_labels_for_scoring(oof)
char_probs = get_char_probs(oof['pn_history'].values,
                            oof[[i for i in range(CFG.max_len)]].values, 
                            CFG.tokenizer)

best_th = 0.5
best_score = 0.
for th in np.arange(0.45, 0.55, 0.01):
    th = np.round(th, 2)
    results = get_results(char_probs, th=th)
    preds = get_predictions(results)
    score = get_score(preds, truths)
    
    if best_score < score:
        best_th = th
        best_score = score
    print(f"th: {th}  score: {score:.5f}")
print(f"best_th: {best_th}  score: {best_score:.5f}")

th: 0.45  score: 0.87919
th: 0.46  score: 0.87894
th: 0.47  score: 0.87896
th: 0.48  score: 0.87916
th: 0.49  score: 0.87918
th: 0.5  score: 0.87927
th: 0.51  score: 0.87926
th: 0.52  score: 0.87923
th: 0.53  score: 0.87924
th: 0.54  score: 0.87919
th: 0.55  score: 0.87937
best_th: 0.55  score: 0.87937


# Data Loading

In [11]:
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features

## make pseudo-label df

In [12]:
test = pd.read_csv(GCS_PATH+'test.csv')
submission = pd.read_csv(GCS_PATH+'sample_submission.csv')
features = pd.read_csv(GCS_PATH+'features.csv')
patient_notes = pd.read_csv(GCS_PATH+'patient_notes.csv')
train_df = pd.read_csv(GCS_PATH+'train.csv')
features = preprocess_features(features)

print(f"test.shape: {test.shape}")
display(test.head())
print(f"features.shape: {features.shape}")
display(features.head())
print(f"patient_notes.shape: {patient_notes.shape}")
display(patient_notes.head())

test.shape: (5, 4)


,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


features.shape: (143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


patient_notes.shape: (42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [13]:
train_pn_num_list=train_df['pn_num'].unique()
s=[]
if debug:
    patient_notes=patient_notes[:10]
for i,row in patient_notes.iterrows():
    if row['pn_num'] not in train_pn_num_list:
        t_features=features[features['case_num']==row['case_num']]
        for j,row_f in t_features.iterrows():
            id=str(row['pn_num']).zfill(5)+'_'+str(row_f['feature_num']).zfill(3)
            s.append([id,row_f['case_num'] , row['pn_num'],row_f['feature_num']])
pl_test=pd.DataFrame(s,columns=test.columns)
pl_test.head()

,id,case_num,pn_num,feature_num
0,00000_000,0,0,0
1,00000_001,0,0,1
2,00000_002,0,0,2
3,00000_003,0,0,3
4,00000_004,0,0,4


In [14]:
test = test.merge(features, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(test.head())

,id,case_num,pn_num,feature_num,feature_text,pn_history
0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [15]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=CFG.seed)
pl_test["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(pl_test["id"], y=pl_test["case_num"])):
    pl_test.loc[valid_idx, "fold"] = fold
pl_test.head()

,id,case_num,pn_num,feature_num,fold
0,00000_000,0,0,0,4
1,00000_001,0,0,1,2
2,00000_002,0,0,2,3
3,00000_003,0,0,3,4
4,00000_004,0,0,4,0


In [16]:
pl_test=pl_test[pl_test['fold'] == CFG.pl_n_fold]
display(pl_test['case_num'].value_counts())

3    30890
5    24512
9    17174
8    14745
4    10610
7     7202
2     6317
0     5637
6     3593
1     1841
Name: case_num, dtype: int64

In [17]:
pl_test = pl_test.merge(features, on=['feature_num', 'case_num'], how='left')
pl_test = pl_test.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(pl_test.head())

,id,case_num,pn_num,feature_num,fold,feature_text,pn_history
0,00000_006,0,0,6,1,Adderall-use,"17-year-old male, has come to the student heal..."
1,00001_004,0,1,4,1,Lightheaded,17 yo male with recurrent palpitations for the...
2,00001_006,0,1,6,1,Adderall-use,17 yo male with recurrent palpitations for the...
3,00001_007,0,1,7,1,Shortness-of-breath,17 yo male with recurrent palpitations for the...
4,00001_011,0,1,11,1,17-year,17 yo male with recurrent palpitations for the...


# Dataset

In [18]:
def prepare_input(cfg, text, feature_text):
    inputs = cfg.tokenizer(text, feature_text, 
                           add_special_tokens=True,
                           max_length=CFG.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
        
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, 
                               self.pn_historys[item], 
                               self.feature_texts[item])
        
        return inputs

# Model

In [19]:
class ScoringModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        
        return output

# Inference

In [20]:
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    
    return predictions

In [21]:
test_dataset = TestDataset(CFG, pl_test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = ScoringModel(CFG, config_path=CFG.config_path, pretrained=False)
    
    state = torch.load(CFG.path+f"{CFG.model.split('/')[1]}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
       
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    prediction = prediction.reshape((len(pl_test), CFG.max_len))
    char_probs = get_char_probs(pl_test['pn_history'].values, prediction, CFG.tokenizer)
    predictions.append(char_probs)
    del model, state, prediction, char_probs
    gc.collect()
    torch.cuda.empty_cache()
    
predictions = np.mean(predictions, axis=0)

  0%|          | 0/3829 [00:00<?, ?it/s]

  0%|          | 0/3829 [00:00<?, ?it/s]

  0%|          | 0/3829 [00:00<?, ?it/s]

  0%|          | 0/3829 [00:00<?, ?it/s]

  0%|          | 0/3829 [00:00<?, ?it/s]

# Submission with Pseudolabels

In [ ]:
results = get_results(predictions, th=0.48)
pl_train=pl_test.copy()
pl_train=pl_train.drop(['feature_text','pn_history'],axis=1)
pl_train['location'] = results
if CFG.NO_ZERO:
    pl_train['location2']=pl_train['location'].str.split(';')
    pl_train=pl_train.fillna({'location2':''})
    pl_train['annotation_length'] = pl_train['location2'].apply(len)
    pl_train=pl_train[pl_train['annotation_length']>0]
    pl_train=pl_train.drop(['annotation_length','location2'],axis=1)
display(pl_train.head())
pl_train.to_csv('pl_train1.csv', index=False)

# For pseudosubmission on Kaggle
#submission['location'] = results
#display(submission.head())
#submission[['id', 'location']].to_csv('pl_train.csv', index=False)

,id,case_num,pn_num,feature_num,fold,location
0,00000_006,0,0,6,1,521 526
1,00001_004,0,1,4,1,179 195
2,00001_006,0,1,6,1,347 353
3,00001_007,0,1,7,1,220 235
4,00001_011,0,1,11,1,1 5


P.S. Run on local machine; (with RTXA4000 GPUs, 8 hours 52 minutes/ ~€2.25 )  